In [89]:
import math
from collections import namedtuple
from random import Random

import pandas as pd

from algorithm_wrappers import concat_placement
from c_interop import Library
from common import Item
from data_analys.data_generator import guillotine_cutting, guillotine_cutting_max
from ortools_models import task_one_model
from problem_io import load_instance, save_instance
from skyline import skyline_decode

In [2]:
# noinspection PyTypeChecker
Instance = namedtuple('Instance', ['pallet_width', 'pallet_height', 'items'])

In [48]:
gen_types = ['random', 'guillotine_rand', 'guillotine_equal', 'guillotine_max']


def generate_instance(w: int, h: int, items_count: int, gen_type: str, seed: int = 1, min_side: int = 5,
                      max_side: int = 20) -> Instance:
    if gen_type == 'random':
        rng = Random(seed)
        return Instance(w, h, [Item(rng.randint(min_side, max_side), rng.randint(min_side, max_side)) for _ in
                               range(items_count)])
    if gen_type.startswith('guillotine'):
        if gen_type == 'guillotine_rand':
            items_xywh = guillotine_cutting(w, h, items_count, min_side, '', seed)
        elif gen_type == 'guillotine_equal':
            items_xywh = guillotine_cutting(w, h, items_count, min_side, seed=seed)
        elif gen_type == 'guillotine_max':
            items_xywh = guillotine_cutting_max(w, h, items_count, min_side, seed)
        else:
            assert False

        items_wh = [(w, h) if w > h else (h, w)
                    for x, y, w, h in items_xywh
                    ]
        items_cl = [Item(int(w), int(h)) for w, h in items_wh]
        return Instance(w, h, items_cl)


def generate_random(items_count: int, seed: int = 1, min_side: int = 5, max_side: int = 20) -> Instance:
    rng = Random(seed)
    items_cl = [Item(rng.randint(min_side, max_side), rng.randint(min_side, max_side)) for _ in range(items_count)]
    area = sum(it.width * it.height for it in items_cl)
    w = round(math.sqrt(area))
    h = round(math.sqrt(area))
    return Instance(w, h, items_cl)

In [5]:
random_instances = [(gen_type, seed, generate_instance(100, 100, n, gen_type, seed)) for gen_type in gen_types for n in
                    [10, 20, 50, 100] for seed in range(1)]

In [51]:
add_random_instances = [('random', seed, generate_random(n, seed)) for n in [10, 20, 50, 100] for seed in [1]]

In [53]:
for gen_type, seed, instance in add_random_instances:
    save_instance(f'data/random/{gen_type}_{len(instance.items)}_{seed}.txt', *instance)

In [13]:
problems = []
for gen_type in gen_types:
    for n in [10, 20, 50, 100]:
        for seed in [0]:
            path = f'data/random/{gen_type}_{n}_{seed}.txt'
            problems.append((gen_type, path, Instance(*load_instance(path))))

In [54]:
problems = []
for gen_type in ['random']:
    for n in [10, 20, 50, 100]:
        for seed in [1]:
            path = f'data/random/{gen_type}_{n}_{seed}.txt'
            problems.append((gen_type, path, Instance(*load_instance(path))))

In [18]:
gens, files, instances = zip(*problems)
dataframe = pd.DataFrame(zip(gens, files), columns=['Generator type', 'File'])

In [67]:
for gen, file, inst in problems:
    alg_count = len(dataframe.columns) - 2
    new_row = pd.Series({'Generator type': gen, 'File': file})
    dataframe = dataframe.append(new_row, ignore_index=True)

In [98]:
dataframe.to_csv('data/results.csv', index=False)

In [108]:
dataframe = pd.read_csv('data/results.csv')
# noinspection PyRedeclaration
instances = [Instance(*load_instance(file)) for file in dataframe['File']]

In [102]:
algorithms = ['Skyline', 'Skyline sorted', 'Concat', 'Annealing']
df_values = ['Solver solution', 'Solver bound'] + algorithms

In [ ]:
# dataframe['Solver solution']=[pd.NA]*len(dataframe)
# dataframe['Solver bound']=[pd.NA]*len(dataframe)
# dataframe['Skyline']=[pd.NA]*len(dataframe)
# dataframe['Skyline sorted']=[pd.NA]*len(dataframe)
# dataframe['Concat']=[pd.NA]*len(dataframe)
# dataframe['Annealing skyline']=[pd.NA]*len(dataframe)

In [85]:
for i in range(len(dataframe)):
    if pd.isnull(dataframe['Solver bound'][i]):
        inst = instances[i]
        print('Computing', i)
        _, obj, bound, _ = task_one_model(inst.pallet_width, inst.pallet_height, inst.items, limit=5 * 60 * 1000,
                                          backend='Gurobi')
        dataframe.at[i, 'Solver solution'] = obj
        dataframe.at[i, 'Solver bound'] = bound

Computing 16
Computing 17
Computing 18
Computing 19


In [80]:
for i in range(len(dataframe)):
    if pd.isnull(dataframe['Skyline'][i]):
        inst = instances[i]
        result, _ = skyline_decode(*inst)
        dataframe.at[i, 'Skyline'] = result

In [82]:
for i in range(len(dataframe)):
    if pd.isnull(dataframe['Skyline sorted'][i]):
        inst = instances[i]
        items = inst.items
        order = list(range(len(items)))
        order.sort(key=lambda j: items[j].height * items[j].width, reverse=True)
        result, _ = skyline_decode(*inst, order)
        dataframe.at[i, 'Skyline sorted'] = result

In [83]:
for i in range(len(dataframe)):
    if pd.isnull(dataframe['Concat'][i]):
        inst = instances[i]
        result, _ = concat_placement(*inst)
        dataframe.at[i, 'Concat'] = result

In [90]:
lib = Library('lib/libcpp.dll')
annealing = lib.simulated_annealing

In [92]:
for i in range(len(dataframe)):
    if pd.isnull(dataframe['Annealing skyline'][i]):
        inst = instances[i]
        power = max(max(it.width * it.height for it in inst.items), inst.pallet_width * inst.pallet_height / 4)
        print('Computing', i)
        result, _ = annealing(*inst, steps=10 ** 6, power=power)
        dataframe.at[i, 'Annealing skyline'] = result

Computing 0
Computing 1
Computing 2
Computing 3
Computing 4
Computing 5
Computing 6
Computing 7
Computing 8
Computing 9
Computing 10
Computing 11
Computing 12
Computing 13
Computing 14
Computing 15
Computing 16
Computing 17
Computing 18
Computing 19


In [99]:
dataframe

,Generator type,File,Solver solution,Solver bound,Skyline,Skyline sorted,Concat,Annealing skyline
0,random,data/random/random_10_0.txt,8288.0,8288.0,8288,8288,8568,8288
1,random,data/random/random_20_0.txt,6540.0,6540.0,6540,6540,6540,6540
2,random,data/random/random_50_0.txt,2181.0,2181.0,2181,2181,2181,2181
3,random,data/random/random_100_0.txt,1734.0,-5285.0,93,29,88,0
4,guillotine_rand,data/random/guillotine_rand_10_0.txt,0.0,0.0,0,0,0,0
5,guillotine_rand,data/random/guillotine_rand_20_0.txt,368.0,0.0,2258,775,187,143
6,guillotine_rand,data/random/guillotine_rand_50_0.txt,897.0,0.0,490,286,1026,50
7,guillotine_rand,data/random/guillotine_rand_100_0.txt,1739.0,0.0,364,431,648,0
8,guillotine_equal,data/random/guillotine_equal_10_0.txt,0.0,0.0,0,0,0,0
9,guillotine_equal,data/random/guillotine_equal_20_0.txt,462.0,0.0,1232,958,1646,0


In [ ]:
def form(x, area):
    if isinstance(x, int) or isinstance(x, float):
        return f'{x / area:.2%}'
    else:
        return x

In [107]:
dataframe.apply(lambda r, ind: r.apply(form, args=(ind[r.name].pallet_width * ind[r.name].pallet_height,)),
                args=(instances,), axis=1)

,Generator type,File,Solver solution,Solver bound,Skyline,Skyline sorted,Concat,Annealing skyline
0,random,data/random/random_10_0.txt,82.88%,82.88%,82.88%,82.88%,85.68%,82.88%
1,random,data/random/random_20_0.txt,65.40%,65.40%,65.40%,65.40%,65.40%,65.40%
2,random,data/random/random_50_0.txt,21.81%,21.81%,21.81%,21.81%,21.81%,21.81%
3,random,data/random/random_100_0.txt,17.34%,-52.85%,0.93%,0.29%,0.88%,0.00%
4,guillotine_rand,data/random/guillotine_rand_10_0.txt,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
5,guillotine_rand,data/random/guillotine_rand_20_0.txt,3.68%,0.00%,22.58%,7.75%,1.87%,1.43%
6,guillotine_rand,data/random/guillotine_rand_50_0.txt,8.97%,0.00%,4.90%,2.86%,10.26%,0.50%
7,guillotine_rand,data/random/guillotine_rand_100_0.txt,17.39%,0.00%,3.64%,4.31%,6.48%,0.00%
8,guillotine_equal,data/random/guillotine_equal_10_0.txt,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
9,guillotine_equal,data/random/guillotine_equal_20_0.txt,4.62%,0.00%,12.32%,9.58%,16.46%,0.00%
